## Data-driven optimization and decision making - Assignment 2
Juha Reinikainen

In [29]:
from scipy.stats import qmc
from desdeo_problem import variable_builder, \
    ScalarConstraint, MOProblem, ScalarObjective
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor, kernels
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.base import TransformerMixin

seed = 42

In [30]:
# construct the problem
# https://www.mathworks.com/help/gads/multiobjective-optimization-welded-beam.html
def F1(x):
    """
    minimize the fabrication cost of the beam
    """
    return 1.10471 * x[:, 0]**2 * x[:, 1] + 0.04811 * x[:, 2] * x[:, 3] * (14.0 + x[:, 1])


P = 6000.0  # max supported load of the beam
L = 14.0  # length of part


def F2(x):
    """
    Minimize the deflection of the beam
    """
    C = 4*(14**3)/(30*10**6)
    return (P / (x[:, 3] * x[:, 2]**3)) * C


objectives = [
    ScalarObjective("cost", F1, maximize=False),
    ScalarObjective("deflection", F2, maximize=False)
]

"""
Welded beam design problem
x[0] the thickness of the welds
x[1] the length of the welds
x[2] the height of the beam
x[3] the width of the beam
"""
var_names = ["thickness", "length", "height", "width"]
lb = [0.125, 0.1, 0.1, 0.125]
ub = [5.0, 10.0, 10.0, 5.0]
initial_values = [0.125, 0.1, 0.1, 0.125]
variables = variable_builder(var_names, initial_values, lb, ub)


def shear_stress(x, y):
    r1 = 1 / np.sqrt(2 * x[:, 0] * x[:, 1])
    R = np.sqrt(x[:, 1]**2 + (x[:, 0] + x[:, 2])**2)
    r2 = ((L + x[:, 1]/2) * R) / np.sqrt(2 * x[:, 0] *
                                         x[:, 2] * ((x[:, 1]**2)/3 + (x[:, 0] + x[:, 2])**2))
    r = P * np.sqrt(r1**2 + r2**2 + (2*r1*r2*x[:, 1])/R)
    return 13600 - r


constraints = [
    ScalarConstraint("thickness", 4, 2, lambda x, y: x[:, 3] - x[:, 0]),
    ScalarConstraint("shear stress", 4, 2, shear_stress),
    ScalarConstraint("normal stress", 4, 2, lambda x,
                     y: 30000 - P * (6 * L)/(x[:, 3]*x[:, 2]**2)),
    ScalarConstraint("buckling load", 4, 2, lambda x, y: (
        64746.022 * (1-0.0282346*x[:, 2])*x[:, 2]*x[:, 3]**3) - 6000),
]

prob = MOProblem(objectives=objectives, variables=variables,
                 constraints=constraints)


In [31]:

# Generate samples and scale them based on the box constraints
lhs = qmc.LatinHypercube(4, seed=seed)
X = lhs.random(400)
X = qmc.scale(X, lb, ub)
res = prob.evaluate(X)
y = res.objectives

# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# y = scaler.fit_transform(y)

# class Objective2Transformer(TransformerMixin):
#     def fit_transform(self, X, y = None, **fit_params):
#         return X[:,2:]

y1 = y[:, 0]
y2 = y[:, 1]
# print(y1.min(), y1.max())
# print(y2.min(), y2.max())
# import matplotlib.pyplot as plt
# plt.scatter([yi for yi,c in zip(y1, res.constraints) if (c >= 0).all()], \
#     [yi for yi,c in zip(y2, res.constraints) if (c >= 0).all()])
# plt.show()
# fix, axs = plt.subplots(2)
# axs[0].hist(y1)
# axs[1].hist(y2)
# plt.show()

# TODO: normalize range of objectives
# TODO: shuffle? approximation error in different areas?
# TODO: ScalarDataObjective
# TODO: approximating areas where solutions are unfeasible is not neccessary
# TODO: find another version of problem description
# TODO: maybe do multiple runs
# TODO: the second objective on dimends on two variables
# TODO: use same score metrics for tuning as crossvalidation?

# test samples
X_test = lhs.random(50)
X_test = qmc.scale(X_test, lb, ub)

res_test = prob.evaluate(X_test)
y_test = res_test.objectives
y1_test = y_test[:, 0]
y2_test = y_test[:, 1]


In [32]:
randomForestRegressor_y1 = RandomForestRegressor()
randomForestRegressor_y1.fit(X, y1)
randomForestRegressor_y2 = RandomForestRegressor()
randomForestRegressor_y2.fit(X[:, 2:], y2)
# randomForestRegressor.fit(X,y)
# sklearn only has negative rmse for some reason
# min rmse, max r2
scoring = ["r2", "neg_root_mean_squared_error"]
# 5-folds
scores_y1 = cross_validate(randomForestRegressor_y1,
                           X, y1, scoring=scoring, cv=5)
scores_y2 = cross_validate(randomForestRegressor_y2,
                           X[:, 2:], y2, scoring=scoring, cv=5)
print("objective 1")
print("r2", scores_y1["test_r2"], "mean", scores_y1["test_r2"].mean(
), "std", scores_y1["test_r2"].std())
print("rmse", scores_y1["test_neg_root_mean_squared_error"],
      "mean", scores_y1["test_neg_root_mean_squared_error"].mean(),
      "std", scores_y1["test_neg_root_mean_squared_error"].std())

print("objective 2")
print("r2", scores_y2["test_r2"], "mean", scores_y2["test_r2"].mean(
), "std", scores_y2["test_r2"].std())
print("rmse", scores_y2["test_neg_root_mean_squared_error"],
      "mean", scores_y2["test_neg_root_mean_squared_error"].mean(),
      "std", scores_y2["test_neg_root_mean_squared_error"].std())


objective 1
r2 [0.966 0.971 0.973 0.972 0.976] mean 0.9716673298402456 std 0.0034609644379274174
rmse [-9.678 -10.321 -8.807 -9.285 -8.588] mean -9.335781605503602 std 0.6211121440238567
objective 2
r2 [0.714 0.555 -4.590 0.980 0.565] mean -0.3554313269698042 std 2.122952476723004
rmse [-33.745 -101.239 -35.096 -7.949 -56.505] mean -46.90686182976308 std 31.223808842493536


In [33]:
parameters = {
    "alpha": [1e-10, 1e-1, 1, 10], #[1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e-0, 1e1, 1e2, 1e3, 1e4],
    #"normalize_y": [True, False],
    "kernel": [
        kernels.RBF(length_scale_bounds=(1e-10, 1000000)),
        kernels.ExpSineSquared(length_scale_bounds=(1e-10, 1000000))
    ]
}

gaussianProcessRegressor_y1 = \
    GridSearchCV(GaussianProcessRegressor(normalize_y=True,
        n_restarts_optimizer=1, random_state=seed), parameters)
gaussianProcessRegressor_y2 = \
    GridSearchCV(GaussianProcessRegressor(normalize_y=True,
        n_restarts_optimizer=1, random_state=seed), parameters)
# fit best parameters
gaussianProcessRegressor_y1.fit(X, y1)
gaussianProcessRegressor_y1 = gaussianProcessRegressor_y1.best_estimator_
gaussianProcessRegressor_y2.fit(X[:, 2:], y2)
gaussianProcessRegressor_y2 = gaussianProcessRegressor_y2.best_estimator_
print(gaussianProcessRegressor_y1)
print(gaussianProcessRegressor_y2)

# 5-folds
scores_y1 = cross_validate(gaussianProcessRegressor_y1,
                           X, y1, scoring=scoring, cv=5)
scores_y2 = cross_validate(gaussianProcessRegressor_y2,
                           X[:, 2:], y2, scoring=scoring, cv=5)

print("objective 1")
print("r2", scores_y1["test_r2"], "mean", scores_y1["test_r2"].mean(
), "std", scores_y1["test_r2"].std())
print("rmse", scores_y1["test_neg_root_mean_squared_error"],
      "mean", scores_y1["test_neg_root_mean_squared_error"].mean(),
      "std", scores_y1["test_neg_root_mean_squared_error"].std())

print("objective 2")
print("r2", scores_y2["test_r2"], "mean", scores_y2["test_r2"].mean(
), "std", scores_y2["test_r2"].std())
print("rmse", scores_y2["test_neg_root_mean_squared_error"],
      "mean", scores_y2["test_neg_root_mean_squared_error"].mean(),
      "std", scores_y2["test_neg_root_mean_squared_error"].std())


c:\Users\Juha Reinikainen\Documents\dataopt\env\lib\site-packages\sklearn\gaussian_process\_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\Juha Reinikainen\Documents\dataopt\env\lib\site-packages\sklearn\gaussian_process\_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\Juha Reinikainen\Documents\dataopt\env\lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified upper bound 1000000.0. Increa

GaussianProcessRegressor(kernel=ExpSineSquared(length_scale=1, periodicity=1),
                         n_restarts_optimizer=1, normalize_y=True,
                         random_state=42)
GaussianProcessRegressor(alpha=0.1,
                         kernel=ExpSineSquared(length_scale=1, periodicity=1),
                         n_restarts_optimizer=1, normalize_y=True,
                         random_state=42)


c:\Users\Juha Reinikainen\Documents\dataopt\env\lib\site-packages\sklearn\gaussian_process\_gpr.py:610: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\Juha Reinikainen\Documents\dataopt\env\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-10. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\Juha Reinikainen\Documents\dataopt\env\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter periodicity is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better val

objective 1
r2 [1.000 1.000 1.000 1.000 1.000] mean 0.9999953044730745 std 1.2140493896679959e-06
rmse [-0.106 -0.115 -0.113 -0.116 -0.148] mean -0.11946919371550106 std 0.014554306896624177
objective 2
r2 [-0.000 0.534 -0.440 -0.003 -0.002] mean 0.01775180655793025 std 0.30887067035507587
rmse [-63.107 -103.614 -17.812 -55.821 -85.690] mean -65.20881028181434 std 29.097886498310366


In [34]:
randomForestRegressor_y1.fit(X, y1)
randomForestRegressor_y2.fit(X[:, 2:], y2)
y1_pred_forest = randomForestRegressor_y1.predict(X_test)
y2_pred_forest = randomForestRegressor_y2.predict(X_test[:, 2:])
print("random forest test")
print("r2 objective 1", r2_score(y1_test, y1_pred_forest))
print("rmse objective 1", mean_squared_error(
    y1_test, y1_pred_forest, squared=False))
print("r2 objective 2", r2_score(y2_test, y2_pred_forest))
print("rmse objective 2", mean_squared_error(
    y2_test, y2_pred_forest, squared=False))

y1_pred_kriging = gaussianProcessRegressor_y1.predict(X_test)
y2_pred_kriging = gaussianProcessRegressor_y2.predict(X_test[:, 2:])
print("kriging test")
print("r2 objective 1", r2_score(y1_test, y1_pred_kriging))
print("rmse objective 1", mean_squared_error(
    y1_test, y1_pred_kriging, squared=False))
print("r2 objective 2", r2_score(y2_test, y2_pred_kriging))
print("rmse objective 2", mean_squared_error(
    y2_test, y2_pred_kriging, squared=False))


random forest test
r2 objective 1 0.9820018939766092
rmse objective 1 8.355749526953662
r2 objective 2 0.9726699928742291
rmse objective 2 3.214175671694415
kriging test
r2 objective 1 0.9999958210149814
rmse objective 1 0.12732316249506315
r2 objective 2 -0.12101709512524295
rmse objective 2 20.585236213432715


|  | | Objective 1 | | Objective 2 | |
| ---- | ---- | -- | -- | -- | -- |
|  | | Kriging | Random forest | Kriging | Random forest |
| k-fold CV | R-squared | | | | |
|  | RMSE | | | | |
| Test | R-squared | | | | |
|  | RMSE | | | | |



In [35]:
# gaussian process inaccurate in large >100 y1 range?
np.set_printoptions(formatter={"all": lambda s: f"{s:.3f}"})
print(np.array([
    randomForestRegressor_y1.predict(X_test[:10]),
    gaussianProcessRegressor_y1.predict(X_test[:10]),
    y1_test[:10]
]))
print(np.array([
    randomForestRegressor_y2.predict(X_test[:10, 2:]),
    gaussianProcessRegressor_y2.predict(X_test[:10, 2:]),
    y2_test[:10]
]))


[[28.330 49.905 64.355 96.606 89.047 35.097 147.490 25.896 2.682 58.826]
 [28.339 58.063 67.037 116.896 83.067 27.356 154.680 29.046 2.234 42.012]
 [28.339 58.311 67.051 116.965 83.068 27.362 154.663 29.033 2.230 42.012]]
[[0.002 33.004 0.018 0.002 0.002 0.532 0.002 0.001 0.271 0.157]
 [10.203 10.203 10.203 10.203 10.203 10.203 10.203 10.203 10.203 10.203]
 [0.002 11.297 0.016 0.001 0.003 0.413 0.002 0.001 0.623 0.113]]
